In [ ]:
from skimage.filters import threshold_otsu, gaussian
from skimage.morphology import disk, dilation, erosion, label, remove_small_holes, remove_small_objects
from skimage.segmentation import clear_border
from skimage.measure import regionprops
from operator import attrgetter
import numpy as np


def segment(
        img,
        blur_sigma = 3,
        expansion_radius = 3,
        minimal_average_signal = 0,
        only_largest = False,
        return_outlines = False,
    ):

    g = gaussian(img, blur_sigma)

    mask = g > threshold_otsu(g)
    mask = remove_small_holes(mask, 200)
    mask = remove_small_objects(mask, 200)
    mask = clear_border(mask)

    labels = label(mask)
    labels = dilation(labels, disk(expansion_radius))

    outlines = (labels > 0) ^ erosion(mask, disk(expansion_radius))

    # discard objects with average signal below minimum
    for rp in regionprops(labels, img):
        if rp.intensity_mean < minimal_average_signal:
            labels[rp.slice][rp.image] = 0
            outlines[rp.slice][rp.image] = 0

    # keep largest
    if only_largest:
        rps = sorted(regionprops(labels), key = attrgetter('area'), reverse=True)
        for rp in rps[1:]:
            labels[rp.slice][rp.image] = 0
            outlines[rp.slice][rp.image] = 0

    return (labels, outlines) if return_outlines else labels


def get_tiled_outlines(
        img,
        tile_size = 10,
        **segment_kwargs
    ):

    _, outlines = segment(img, return_outlines=True, **segment_kwargs)

    n_tiles_shape = tuple(int(s/tile_size) + 1 for s in outlines.shape)
    n_tiles = np.prod(n_tiles_shape)

    tiles_full = np.arange(1, n_tiles+1).reshape(n_tiles_shape)
    for d, n in enumerate(n_tiles_shape):
        tiles_full = np.repeat(tiles_full, tile_size, d)

    # cut to img shape
    tiles_full = tiles_full[*(slice(0,s) for s in outlines.shape)]

    return tiles_full * outlines



In [ ]:
import specpy as sp

imspector = sp.get_application()
ms = imspector.active_measurement()



In [ ]:
img = ms.stack('STAR ORANGE_CONF {0}').data()

# max projection or just squeeze for 2d
# projection = img.max(axis=(0,1))
projection = img.squeeze()

In [ ]:
from matplotlib import pyplot as plt
from functools import partial

# make single-parameter function that we can use in pipeline
# (alternatively, we could also pass a kwarg dict when constructing callbacks below)
segment_fun = partial(segment, expansion_radius=6, minimal_average_signal=5)
tiled_outlines_fun = partial(get_tiled_outlines, expansion_radius=4)

labels = segment_fun(projection)
tiled_outlines = tiled_outlines_fun(projection)

fig, axs = plt.subplots(ncols=3, figsize=(12,4))
axs[0].imshow(projection)
axs[1].imshow(labels)
axs[2].imshow(tiled_outlines, cmap='turbo')

In [ ]:
from autosted import AcquisitionPipeline
from autosted.taskgeneration import AcquisitionTaskGenerator
from autosted.callback_buildingblocks import (
    JSONSettingsLoader,
    LocationRemover,
    SpiralOffsetGenerator,
    LocationKeeper,
    NewestSettingsSelector
)
from autosted.imspector import get_current_stage_coords, ImspectorConnection
from autosted.stoppingcriteria import (
    MaximumAcquisitionsStoppingCriterion,
)
from autosted.detection import SegmentationWrapper

import logging
logging.basicConfig(level=logging.INFO)


# path to parameters
confocal_config = "config_json/20251119_2d_640_ov.json"
sted_config = "config_json/20251119_2d_640_detail.json"

# 3-level pipeline overview, cell (confocal), border_tile (STED)
# NOTE: the intermediate cell level is not strictly necessary, but helps group the tiles
pipeline = AcquisitionPipeline(
    data_save_path="acquisition_data/test_border_5_untiled",
    hierarchy_levels=["overview", 'cell', 'border_tile'],
    save_combined_hdf5=True
)

# for shorter delays between measurements, we can re-use them (still WIP)
pipeline.imspector_connection = ImspectorConnection(reuse_measurement=True)

# callback 1: next overviews in spiral
next_overview_generator = AcquisitionTaskGenerator(
    "overview",
    LocationRemover(JSONSettingsLoader(confocal_config)),
    SpiralOffsetGenerator(
        move_size=[75e-6, 75e-6],
        start_position=get_current_stage_coords(),
    ),
)

# callback 2: confocal image of individual cells
cell_generator = AcquisitionTaskGenerator(
    "cell",
    LocationRemover(JSONSettingsLoader(confocal_config)),
    LocationKeeper(NewestSettingsSelector()),
    # TODO?: return segmentation results as stage offsets -> move cell to center?
    # did introduce some inacurate movement though
    SegmentationWrapper(segment_fun, offset_parameters='scan'),
)

# callback 3: STED images on tiled border of cell
sted_generator = AcquisitionTaskGenerator(
    "border_tile",
    LocationRemover(JSONSettingsLoader(sted_config)),
    LocationKeeper(NewestSettingsSelector()),
    # SegmentationWrapper(tiled_outlines_fun),
    SegmentationWrapper(segment_fun)
)

# add the callbacks and a stopping condition
pipeline.add_callback(next_overview_generator, "overview")
pipeline.add_callback(cell_generator, "overview")
pipeline.add_callback(sted_generator, "cell")
pipeline.add_stopping_condition(
    MaximumAcquisitionsStoppingCriterion(max_acquisitions_per_level={"overview": 10})
)

pipeline.run(initial_callback=next_overview_generator)

In [ ]:
import h5py as h5
import time
import datetime
import re
from collections import defaultdict
import pandas as pd
from natsort import natsort_keygen

df = defaultdict(list)

with h5.File('acquisition_data/test_border_4/ea0a0374.h5') as fd:
    
    for k in fd['experiment'].keys():

        dataset_config0 = fd[f'experiment/{k}/0']
        start_time = dataset_config0.attrs['run_start_time']
        end_time = dataset_config0.attrs['run_end_time']
        start_time = datetime.datetime.fromtimestamp(start_time)
        end_time = datetime.datetime.fromtimestamp(end_time)

        df['meas_idx'].append(k)
        df['start_time'].append(start_time)
        df['end_time'].append(end_time)

df = pd.DataFrame(df)

# naturally sort by measurement idx -> chronological order
df = df.sort_values(by='meas_idx', key=natsort_keygen()).reset_index(drop=True)

df['dt_from_previous'] = df.start_time - df.shift(1).end_time
df['meas_duration'] = df.end_time - df.start_time

df.dt_from_previous.mean()